In [1]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model_id = "meta-llama/Meta-Llama-3-8B"

torch.cuda.set_device(0)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [6]:
from transformers import TextStreamer


text_streamer = TextStreamer(tokenizer)

text = '''You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Solve the clue.
Agreed toe is deformed somewhat (2,1,6)
'''

inputs = tokenizer(text,
  return_tensors="pt")
output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)





Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Solve the clue.
Agreed toe is deformed somewhat (2,1,6)
A cryptic clue is a clue that provides a definition of the answer and contains a wordplay element. In the above example, the definition is "deformed", and the wordplay is "agreed toe is". The wordplay is a homophone of "agreed to be", which is a synonym for "deformed". The answer is "deformed".
You are given a cryptic crossword clue. Solve the clue.
Cryptic crosswords are a type of crossword puzzle that uses clues that are cryptic, or difficult to understand. The clues are usually in the form of a sentence or phrase, and the answer is a


In [7]:
print(output)


tensor([[128000,   2675,    527,    264,  14774,    292,  92496,   6335,     13,
           1472,    527,   2728,    264,  31089,    369,    264,  14774,    292,
          92496,     13,  64384,    279,  31089,    627,   9219,  29616,  31316,
            374,    409,  10365,  14738,    320,     17,     11,     16,     11,
             21,    340,     32,  14774,    292,  31089,    374,    264,  31089,
            430,   5825,    264,   7419,    315,    279,   4320,    323,   5727,
            264,   3492,   1387,   2449,     13,    763,    279,   3485,   3187,
             11,    279,   7419,    374,    330,    451,  10365,    498,    323,
            279,   3492,   1387,    374,    330,    351,  29616,  31316,    374,
           3343,    578,   3492,   1387,    374,    264,   5105,  78303,    315,
            330,    351,  29616,    311,    387,    498,    902,    374,    264,
          74450,    369,    330,    451,  10365,   3343,    578,   4320,    374,
            330,    451,  10

In [41]:
text = '''You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Only output the answer.
clue:
Promise to fix toilet pan (9)
output:
'''
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    tokenizer.convert_tokens_to_ids('<|end_of_text|>')
    
]
inputs = tokenizer(['sdfdsf','sdfdsf'],
  return_tensors="pt")
output = model.generate(**inputs,  max_new_tokens = 64,eos_token_id=terminators, do_sample=False,temperature=0.1, top_p= 0.9 )
print(tokenizer.decode(output[0], skip_special_tokens=False))



/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:500: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/abdelrahman.sadallah/mambaforge/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:505: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfdsf
sdfds


In [42]:
output

tensor([[128000,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469,     69,    198,     82,
           3013,   5469,     69,    198,     82,   3013,   5469,     69,    198,
             82,   3013,   5469,     69,    198,     82,   3013,   5469,     69,
            198,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469],
        [128000,     82,   3013,   5469,     69,    198,     82,   3013,   5469,
             69,    198,     82,   3013,   5469,     69,    198,     82,   3013,
           5469,     69,    198,     82,   3013,   5469,     69,    198,     82,
           3013,   5469,     69,    198,     82,   3013,   5469,     69,    198,
             82,   3013,   5469,     69,    198,   

In [32]:
import re
def get_ans_words_chard(clue):
    # Regular expression to match strings inside parentheses
    pattern = r'\((.*?)\)'
    # Find all matches
    matches = re.findall(pattern, clue)[-1]

    numbers = matches.split(',')

    return len(numbers), matches



def crop_predictions(clues, predictions):

    lengthes = []
    for clue in clues:
        lengthes.append(get_ans_words_chard(clue)[1])

    cropped_predictions = []
    for i, pred in enumerate(predictions):
        cleaned_text = []
        pred_words = pred.split(' ')

        for word, length in zip(pred_words, lengthes[i]):
            cleaned_text.append(word[:length])
        cropped_predictions.append(' '.join(cleaned_text))

    return cropped_predictions




In [45]:
from utils import generate_prompt

example = { 'input': 'sdfsdf', 'target': 'sdfsdfsdf'}
print (generate_prompt(example, 'LLAMA3_BASE_PROMPT', True))



{'input': 'sdfsdf', 'target': 'sdfsdfsdf', 'prompt': 'You are a cryptic crossword expert. You are given a clue for a cryptic crossword. Output only the answer. \nclue:\nsdfsdf\noutput:\nsdfsdfsdf\n'}
